## Hypergraph


This notebook demonstrates the JavaScript implementation of the hypergraph transformation. The transformation finds relationships between entities ( unique values of each `DataFrame` column) by linking them through their shared rows. This affords construction of a graph from any structured high-dimensional dataset, i.e. cyber security event logs. (The simple example here was inspired by this [Graphistry blog post](https://www.graphistry.com/blog/no-code-file-uploader-hypergraph).)

First let's import the relevant classes and functions from ***@rapidsai***

In [6]:
var {Series, DataFrame} = require("@rapidsai/cudf")
var {hypergraphDirect} = require("@rapidsai/cugraph")

Now let's construct a `DataFrame` that represents events from a network log. In this case suppose someone from host **128.0.0.1** is scaning hosts **172.0.0.2** and **172.0.0.3** for vulnerability **cve-123**.

In [3]:
var log = new DataFrame({
    src_ip: ['128.0.0.1', '128.0.0.1'], 
    dest_ip: ['172.0.0.2', '172.0.0.3'],
    vuln: ['cve-123', 'cve-123']
})

Now we can call `hypergraphDirect` on this `DataFrame` to generate a graph of inferred relationships:

In [7]:
var h = hypergraphDirect(log)

The `nodes` property shows all the entities inferred from the original log. In this case we see all of the host IPs as well as the specific vulnerability show up in the `node_id` column:

In [8]:
console.log(h.nodes.toString())

  dest_ip    src_ip    vuln            node_id node_type category
172.0.0.2      null    null dest_ip::172.0.0.2   dest_ip  dest_ip
172.0.0.3      null    null dest_ip::172.0.0.3   dest_ip  dest_ip
     null 128.0.0.1    null  src_ip::128.0.0.1    src_ip   src_ip
     null      null cve-123      vuln::cve-123      vuln     vuln



All of the inferred relationships show up in the `edges` property:

In [10]:
console.log(h.edges.toString())

   event_id       edge_type                src               dst   dest_ip    src_ip    vuln
event_id::0 dest_ip::src_ip dest_ip::172.0.0.2 src_ip::128.0.0.1 172.0.0.2 128.0.0.1 cve-123
event_id::1 dest_ip::src_ip dest_ip::172.0.0.3 src_ip::128.0.0.1 172.0.0.3 128.0.0.1 cve-123
event_id::0   dest_ip::vuln dest_ip::172.0.0.2     vuln::cve-123 172.0.0.2 128.0.0.1 cve-123
event_id::1   dest_ip::vuln dest_ip::172.0.0.3     vuln::cve-123 172.0.0.3 128.0.0.1 cve-123
event_id::0    src_ip::vuln  src_ip::128.0.0.1     vuln::cve-123 172.0.0.2 128.0.0.1 cve-123
event_id::1    src_ip::vuln  src_ip::128.0.0.1     vuln::cve-123 172.0.0.3 128.0.0.1 cve-123



Examining this output, we see:

* the **172.0.0.*** hosts are both related to the **128.0.0.1** host  
* the **172.0.0.*** hosts are both related to the vulnerability **cve-123**
* the **128.0.0.1** host is related to the vulnerability **cve-123** in two different ways

In [12]:
console.log(h.entities.toString())

  dest_ip    src_ip    vuln            node_id node_type category
172.0.0.2      null    null dest_ip::172.0.0.2   dest_ip  dest_ip
172.0.0.3      null    null dest_ip::172.0.0.3   dest_ip  dest_ip
     null 128.0.0.1    null  src_ip::128.0.0.1    src_ip   src_ip
     null      null cve-123      vuln::cve-123      vuln     vuln

